# 1. Package Loading

In [ ]:
# Standard Library
import json
import os
import re
import requests

# Data Manipulation
import pandas as pd
import numpy as np

# PDFs
import pdfplumber

# LLMs
from openai import OpenAI
import tiktoken
import textwrap

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.graphs.graph_document import (
    GraphDocument,
    Node as BaseNode,
    Relationship as BaseRelationship,
)
from langchain_huggingface import HuggingFaceEmbeddings

# Neo4j
from neo4j import GraphDatabase
from neo4j.exceptions import AuthError

# Typing & Validation
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional

# 2. Raw Data Extraction

## 2.1 Incident Reports

In [2]:
# Define the directory containing PDFs
pdf_directory = "./reports_ie/"

# List all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith(".pdf")]

for file in pdf_files:
    print(file)

IE-10375 - 210827 Collision with track equipment.pdf
IE-10397 - 211207 Clontarf.pdf
IE-10404 - 230222 Broken Rail Emly.pdf
IE-200608 BnM Collision LC Offaly.pdf
IE-6218-200111 Collision RRME Rosslare.pdf
IE-6262-200429 LC Collision XM240.pdf
IE-6291-200524 LC XA068 Ashfield.pdf
IE-6305 - 200707_locomotive_224.pdf


In [3]:
# Specify file name that you want to process
pdf_name = "IE-6218-200111 Collision RRME Rosslare.pdf"

In [4]:
# Define regex patterns to identify the start of the summary and contents sections
SUMMARY_PATTERN = re.compile(r"^summary", re.IGNORECASE)
CONTENTS_PATTERN = re.compile(r"^contents", re.IGNORECASE)

# Define function to extract the summary section from Ireland reports
def extract_summary_section(pdf_path: str, header_lines: int = 1) -> str:
    """
    Extracts text from 'Summary' to 'Contents' in an Irish rail report PDF.
    If no summary section is found, returns the text from all pages.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"File not found: {pdf_path}")

    summary_text = ""
    full_text = ""
    capturing = False  

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text_lines = page_text.split("\n")
                text_without_header = text_lines[header_lines:]
                page_content = f"[Page {page.page_number}]\n" + "\n".join(text_without_header) + "\n\n"

                # Always append to full_text
                full_text += page_content

                if text_without_header:
                    first_line = text_without_header[0].strip().lower()

                    if SUMMARY_PATTERN.match(first_line):
                        capturing = True
                    
                    if CONTENTS_PATTERN.match(first_line) and capturing:
                        # Stop capturing when "Contents" is found after summary started.
                        break
                    
                    if capturing:
                        summary_text += page_content

    if not summary_text:
        print(f"Warning: No summary section found in {pdf_path}. Returning full text.")
        return full_text

    return summary_text

# Example usage:
pdf_text = extract_summary_section(f"./reports_ie/{pdf_name}", header_lines=1)
print(pdf_text[:500])

[Page 4]
Summary
On 11th January 2020, T3 Possessions were granted between the 112 Mile Post (MP) 880
yards and the 112 MP 1320 yards (Rosslare Strand to Rosslare Europort) for Iarnród Éireann’s
Infrastructure Manager’s (IÉ-IM) Chief Civil Engineer’s (CCE) Department in relation to the
Rosslare Coastal Erosion Project.
The work being undertaken in the T3 Possessions included the erection of viewing distance
marker boards (sometimes referred to as V Boards) on the north and south sides of a newly


## 2.2 Accident Categories

In [6]:
# Load category A events
cat_a_events = pd.read_csv("category-a-event-types-source.csv", encoding='latin-1')
cat_a_events.head(5)

,Id,Code,Name,Definition,Broader
0,A1,A1,Collisions,A collision event falling within the A.1 sub-c...,A
1,A1-1,A1.1,Collision of train with a train/rail vehicle,A front to front; front to end or a side colli...,A1
2,A1-2,A1.2,Collision of train with obstacle within the cl...,A collision between a part of a train and obje...,A1
3,A1-3,A1.3,Collision of one or more rail vehicles with an...,Same as A1.1 but concerning more rail vehicles...,A1
4,A1-4,A1.4,Collision of one or more rail vehicles with ob...,Same as A1.2 but concerning one or more rail v...,A1


# 3. Langchain Chunk Splitting

## 3.1 Incident Reports

In [ ]:
# Define function to split the incident report into chunks
def split_report_into_chunks(text: str, chunk_size: int = 2000, chunk_overlap: int = 300) -> list[str]:
    """
    Splits text into smaller overlapping chunks using LangChain's text splitter.
    """
    if not text:
        print("Warning: No text provided for splitting.")
        return []

    if len(text) <= chunk_size:
        return [text]  # If text is smaller than chunk size, return as single chunk

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

# Split the extracted text
report_chunks = split_report_into_chunks(pdf_text)

# Print the number of chunks and a sample chunk
print(f"Total chunks: {len(report_chunks)}\nFirst chunk:\n{report_chunks[0]}")

Total chunks: 4
First chunk:
[Page 4]
Summary
On 11th January 2020, T3 Possessions were granted between the 112 Mile Post (MP) 880
yards and the 112 MP 1320 yards (Rosslare Strand to Rosslare Europort) for Iarnród Éireann’s
Infrastructure Manager’s (IÉ-IM) Chief Civil Engineer’s (CCE) Department in relation to the
Rosslare Coastal Erosion Project.
The work being undertaken in the T3 Possessions included the erection of viewing distance
marker boards (sometimes referred to as V Boards) on the north and south sides of a newly
re-opened level crossing, adjacent to Hayesland IÉ-IM compound. There were four members
of CCE staff, working in pairs, erecting the V Boards on the north and south sides of the level
crossing. The pair working on the south side included the Person In Charge Of Possession
(PICOP)/ Engineering Supervisor (PICOP/ES) and the other pair, working on the north side,
included the Acting Permanent Way Inspector (APWI).
The APWI decided to transport the V Boards using a piec

## 3.2 Accident Categories

In [ ]:
# Convert the DataFrame to a list of rows in dictionary format
cat_a_events_dict = cat_a_events.to_dict("records")
cat_a_events_dict[:5]

[{'Id': 'A1',
  'Code': 'A1',
  'Name': 'Collisions',
  'Definition': 'A collision event falling within the A.1 sub-categories for which detailed information is not (yet) available.',
  'Broader': 'A'},
 {'Id': 'A1-1 ',
  'Code': 'A1.1 ',
  'Name': 'Collision of train with a train/rail vehicle',
  'Definition': 'A front to front; front to end or a side collision between a part of a train and a part of another train or rail vehicle; or with shunting rolling stock.',
  'Broader': 'A1'},
 {'Id': 'A1-2',
  'Code': 'A1.2',
  'Name': 'Collision of train with obstacle within the clearance gauge',
  'Definition': 'A collision between a part of a train and objects fixed or temporarily present on or near the track (except at level crossings if lost by a crossing vehicle or user); including collision with overhead contact lines.',
  'Broader': 'A1'},
 {'Id': 'A1-3 ',
  'Code': 'A1.3 ',
  'Name': 'Collision of one or more rail vehicles with another rail vehicle',
  'Definition': 'Same as A1.1 but 

In [13]:
# Define function to split the event data into chunks
def split_events_into_chunks(data: list[dict], chunk_size: int = 2000, chunk_overlap: int = 300) -> list[list[str]]:
    """
    Converts each dictionary row into a string and splits each string into smaller overlapping chunks.
    """
    if not data:
        print("Warning: No data provided for splitting.")
        return []

    # Prepare the list of formatted strings (chunks) from the dictionary rows
    chunk_list = [
        # f"Id: {row['Id']} Code: {row['Code']} Name: {row['Name']} Definition: {row['Definition']} Broader: {row['Broader']}"
        f"Accident Type: {row['Name']}"
        for row in data
    ]

    # Function to split a single text into smaller chunks
    def split_single_text(text: str) -> list[str]:
        if not text:
            print("Warning: No text provided for splitting.")
            return []

        if len(text) <= chunk_size:
            return [text]  # If text is smaller than chunk size, return as a single chunk

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap
        )
        return text_splitter.split_text(text)

    # Split each chunk string from the dictionary rows into smaller chunks
    all_chunks = [split_single_text(chunk) for chunk in chunk_list]
    
    return all_chunks

event_chunks = split_events_into_chunks(cat_a_events_dict, chunk_size=2000, chunk_overlap=300)

# Print the number of chunks and a sample chunk
print(f"Total chunks: {len(event_chunks)}\nFirst three chunks:\n{event_chunks[:3]}")

Total chunks: 32
First three chunks:
[['Accident Type: Collisions'], ['Accident Type: Collision of train with a train/rail vehicle'], ['Accident Type: Collision of train with obstacle within the clearance gauge']]


# 4. Relevant Chunk Retrieval

In [14]:
# Define embeddings 
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

d:\Software\Miniconda\envs\master_thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4.1 Incident Reports

In [15]:
# Store text chunks into FAISS vector store
vectorstore = FAISS.from_texts(report_chunks, embeddings)

In [16]:
# Define entities of interest that you'd like to extract chunks for from the vector store
entities_of_interest = ["unique accident", "accident type", "track section", "date", "time", "country", "regulatory body"]

# Function for extracting most relevant chunks from vector store
def find_most_relevant_chunks(entities: list[str], top_k: int) -> str:
    """
    Finds the most relevant text chunks for each entity of interest
    using FAISS similarity search and removes duplicates (if same chunk retrieved).
    
    Args:
    - entities (list): List of entity names to query (e.g., ["date", "location"])
    - top_k (int): Number of chunks to retrieve per entity
    
    Returns:
    - unique_relevant_chunks (list): Deduplicated relevant chunks
    """
    retrieved_chunks = set()  # Use a set to avoid duplicate chunks

    for entity in entities:
        print(f"Searching for entity: {entity}")
        query = f"Report details about {entity}."
        found_chunks = vectorstore.similarity_search(query, k=top_k)

        for chunk in found_chunks:
            retrieved_chunks.add(chunk.page_content)  # Add chunk if not already present

    # Convert set back to a list and join into a single string
    unique_relevant_chunks = list(retrieved_chunks)
    combined_text = "\n".join(unique_relevant_chunks)

    print(f"\nFound {len(unique_relevant_chunks)} unique relevant chunks.")
    return combined_text

# Find & combine relevant chunks
relevant_report_text = find_most_relevant_chunks(entities_of_interest, top_k=3)

print(f"\nMost Relevant Chunks Combined:\n{relevant_report_text}")

Searching for entity: unique accident
Searching for entity: accident type
Searching for entity: track section
Searching for entity: date
Searching for entity: time
Searching for entity: country
Searching for entity: regulatory body

Found 4 unique relevant chunks.

Most Relevant Chunks Combined:
[Page 6]
The RAIU make the following safety recommendations:
• 202004-01 – IÉ-IM should classify and define RMMEs, Trolleys, LMEs and other
commonly used plant or equipment on the railway and ensure appropriate safety
procedures are in place for their use. IÉ-IM should also assess the need for any associated
training and competency related to these changes and if considered necessary prepare
training and competency assessment material;
• 202004-02 – IÉ-IM CCE should ensure that, once defined and classified, change
management systems are put in place to ensure RMMEs, Trolleys, LMEs, etc are not
altered for other uses, without first having been safety validated in line with company
processes;
• 2

## 4.2 Accident Categories

In [20]:
# Flatten the list of lists with event chunks for FAISS processing
flat_event_chunks = [chunk for sublist in event_chunks for chunk in sublist]

# Store text chunks into FAISS vector store
vectorstore = FAISS.from_texts(flat_event_chunks, embeddings)

In [21]:
# Define the accident type to search for
accident_type = "Collision"

# Function for extracting most relevant chunks from vector store
def find_most_relevant_chunks(top_k: int) -> str:
    """
    Retrieves the most relevant accident category chunks from the vector store based on the query.
    """
    query = f"{accident_type}"
    
    # Perform the similarity search
    found_chunks = vectorstore.similarity_search(query, k=top_k)
    
    # Extract the text from each document in the list
    found_chunks = [doc.page_content for doc in found_chunks]
    
    # Join the list of texts into a single string
    found_chunks = "\n".join(found_chunks)
    
    return found_chunks

# Find & combine relevant chunks
relevant_events_text = find_most_relevant_chunks(top_k=5)

print(f"{relevant_events_text}")

Accident Type: Collisions
Accident Type: Other collision type
Accident Type: Collision of train with a train/rail vehicle
Accident Type: Collision of one or more rail vehicles with another rail vehicle
Accident Type: Collision of one or more rail vehicles with obstacle within the clearance gauge


# 5. Instantiating Language Models

## 5.1 Instantiating GPT

In [22]:
# Set the API key and model name
MODEL_GPT = "gpt-4o-mini"

# Load OpenAI API Key from requirements file
with open("gpt-personal-key.txt", "r") as file:
    OPENAI_API_KEY = file.read().strip()

# Instantiate OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

### 5.1.1 Token Count

In [23]:
# Function for calculating tokens
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    """Efficiently counts tokens in a text for a given OpenAI model."""
    if model not in count_tokens.encoders:
        count_tokens.encoders[model] = tiktoken.encoding_for_model(model)
    return len(count_tokens.encoders[model].encode(text))

count_tokens.encoders = {}

### 5.1.2 Entity Extraction

In [24]:
# Define classes for the entities extraction
class Property(BaseModel):
    """A single property consisting of key and value."""
    key: str = Field(..., description="Property key")
    value: str = Field(..., description="Property value")

class Node(BaseNode):
    """Represents an entity in the railway accident knowledge graph."""
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    """Represents a relationship between two entities in the graph."""
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """A knowledge graph storing railway accident data."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

### 5.1.3 Building the prompt

In [ ]:
# Define function to build the prompt using the extracted text and other relevant variables
def build_gpt_prompt(text):
    """
    Constructs a structured prompt to extract entities and relationships for railway accidents.
    """
    
    schema_example = """
    {
        "nodes": [
            {"id": "Dublin-Cork Accident", "type": "UniqueAccident"},
            {"id": "Level Crossing Accident involving a train", "type": "AccidentType"},
            {"id": "105 MP-108 MP", "type": "TrackSection"},
            {"id": "23/12/2021", "type": "Date"},
            {"id": "16:32", "type": "Time"},
            {"id": "Ireland", "type": "Country"},
            {"id": "European Rail Agency", "type": "RegulatoryBody"}
        ],
        "rels": [
            {"source": "Dublin-Cork Accident", "target": "Ireland", "type": "occurred_in"},
            {"source": "Dublin-Cork Accident", "target": "Collision", "type": "is_type"},
            {"source": "Dublin-Cork Accident", "target": "105 MP-108 MP", "type": "is_track_section"},
            {"source": "Dublin-Cork Accident", "target": "European Rail Agency", "type": "investigated_by"},
            {"source": "Dublin-Cork Accident", "target": "23/12/2021", "type": "has_date"},
            {"source": "Dublin-Cork Accident", "target": "16:32", "type": "has_time"}
        ]
    }
    """

    return f"""
    Analyze the following railway accident report context and extract structured knowledge in JSON format.

    Return a JSON object with:
    - `nodes`: A list of entities, specifically {entities_of_interest}.
    - `rels`: A list of relationships linking entities.

    Guidelines:
    - Look at the JSON schema example response and follow it closely. 
    - Ensure that the `source` and `target` nodes in `rels` are the SAME entities from the `nodes` list, and NOT different ones. 
    - Make sure to map ALL nodes with other important entities, e.g., (node UniqueAccident has_date node Date, node UniqueAccident occurred_at node Country).
    - Do NOT map entities like (node Date is_date to node Time) or (node AccidentType is_type to node Country). This is INCORRECT.
    - The `type` field in `rels` should be a verb phrase (e.g., "occurred_in", "investigated_by").
    - The `id` field in `nodes` should be the exact text of the entity, not a description or a summary.
    - Pay attention to date and type formats (e.g., EU date format, 24-hour time).
    - The `UniqueAccident` entity should be a unique identifier for the accident.
    - The `AccidentType` entity should be the type of accident. Choose the most relevant one from the list of category events below, and copy it word for word.
    - The `TrackSection` entity should be the track section where the accident occurred.
    - The `Country` entity should be the country where the accident occurred.
    - The `RegulatoryBody` entity should be the regulatory body that investigated the accident.

    Schema example:
    {schema_example}

    Accident type:
    {relevant_events_text}

    Accident report context:
    {text}

    JSON:
    """

### 5.1.4 Calling GPT

In [25]:
# Define function to call GPT model with respective prompt
def call_gpt(prompt, temperature=1):
    """
    Calls the GPT model with the structured prompt and returns the raw response.
    """
    completion = client.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": "You are an expert in analyzing railway accident reports. Return output in JSON format only."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    
    response_text = completion.choices[0].message.content.strip()
    response_text = re.sub(r'^```json\n?|```$', '', response_text).strip()
    
    return response_text

### 5.1.5 Extracting entities

In [27]:
# Define the function to run the GPT model
def extract_entities(text: str) -> Dict[str, Any]:
    """
    Extracts entities & relationships from a railway accident report using GPT.
    If function fails, saves the raw response to a file for review.
    """

    # Build prompt
    prompt = build_gpt_prompt(text)

    # Call GPT
    response_text = call_gpt(prompt)

    try:
        extracted_graph = json.loads(response_text)  # Ensure valid JSON
        return extracted_graph  # Successfully parsed knowledge graph
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", str(e))
        print("Storing raw response for review...")

        # Save the faulty response for debugging
        with open("failed_graph_extractions.json", "a") as file:
            json.dump({"input_text": text[:1000], "raw_output": response_text}, file, indent=4)
            file.write("\n")

        return {}  # Return empty dictionary in case of failure

### 5.1.6 Token approval & execution

In [28]:
# Define token limit for function execution
token_limit = 4096

# Build the prompt and count tokens
prompt = build_gpt_prompt(relevant_report_text)
token_count = count_tokens(prompt)
estimated_cost = token_count * 0.00000015  # Approximate OpenAI pricing

# Check token limit
print(f"Estimated cost: ${estimated_cost:.5f}")
if token_count > token_limit:
    print(f"Token count is too high: {token_count}\nPlease reduce the chunk size or refine the prompt.")
else:
    print(f"Token count for prompt: {token_count}")

Estimated cost: $0.00033
Token count for prompt: 2197


In [29]:
# Confirm execution
proceed = input("Do you want to proceed with information extraction? (yes/no): ").strip().lower()
if proceed != "yes":
    print("Extraction aborted by user.")
else:
    print("Sending request to GPT...")
    gpt_response = extract_entities(relevant_report_text)
    response_dict = {"model": MODEL_GPT, "response": gpt_response}
    
gpt_response

Sending request to GPT...


{'nodes': [{'id': 'Train A602 Collision', 'type': 'UniqueAccident'},
  {'id': 'Collision of one or more rail vehicles with obstacle within the clearance gauge',
   'type': 'AccidentType'},
  {'id': '112 MP 880 yards to 112 MP 1320 yards', 'type': 'TrackSection'},
  {'id': '11/01/2020', 'type': 'Date'},
  {'id': '10:52', 'type': 'Time'},
  {'id': 'Ireland', 'type': 'Country'},
  {'id': 'Railway Accident Investigation Unit', 'type': 'RegulatoryBody'}],
 'rels': [{'source': 'Train A602 Collision',
   'target': 'Ireland',
   'type': 'occurred_in'},
  {'source': 'Train A602 Collision',
   'target': 'Collision of one or more rail vehicles with obstacle within the clearance gauge',
   'type': 'is_type'},
  {'source': 'Train A602 Collision',
   'target': '112 MP 880 yards to 112 MP 1320 yards',
   'type': 'is_track_section'},
  {'source': 'Train A602 Collision',
   'target': 'Railway Accident Investigation Unit',
   'type': 'investigated_by'},
  {'source': 'Train A602 Collision',
   'target': 

## 5.2 Instantiating Local Models

### 5.2.1 Building the prompt

In [ ]:
# Define the function to build the prompt for local models
def build_local_prompt(text):
    """
    Constructs a structured prompt to extract entities and relationships for railway accidents.
    """
    
    schema_example = """
    {
        "nodes": [
            {"id": "Dublin-Cork Accident", "type": "UniqueAccident"},
            {"id": "Train Derailment", "type": "AccidentType"},
            {"id": "23/12/2021", "type": "Date"},
            {"id": "16:32", "type": "Time"},
            {"id": "Ireland", "type": "Country"},
            {"id": "European Rail Agency", "type": "RegulatoryBody"}
        ],
        "rels": [
            {"source": "Dublin-Cork Accident", "target": "Ireland", "type": "occurred_in"},
            {"source": "Dublin-Cork Accident", "target": "Collision", "type": "is_type"},
            {"source": "Dublin-Cork Accident", "target": "European Rail Agency", "type": "investigated_by"},
            {"source": "Dublin-Cork Accident", "target": "23/12/2021", "type": "has_date"},
            {"source": "Dublin-Cork Accident", "target": "16:32", "type": "has_time"}
        ]
    }
    """

    return f"""
    Extract structured entities and output ONLY a JSON object from this railway accident report. Do not provide a summary or comment on the incident.

    Return the JSON object with:
    - `nodes`: A list of entities, specifically {entities_of_interest}.
    - `rels`: A list of relationships linking entities.

    Schema example:
    {schema_example}

    Accident report context:
    {text}

    JSON:
    """

### 5.2.2 Defining the endpoint

In [ ]:
# Define the API URL
url = "http://llama-max-ollama.ai.wu.ac.at/api/generate"

# Define prompt
prompt = build_local_prompt(relevant_report_text)

# Specify local model
MODEL_LOCAL = "llama3.1:latest"

### 5.2.3 Querying the model

In [ ]:
# Define the payload and query the local model
payload = {
    "model": f"{MODEL_LOCAL}",  # Ensure correct model name
    "prompt": f"{prompt}",
    "stream": False  # If 'raw' is unnecessary, remove it
}

# Set headers
headers = {"Content-Type": "application/json"}

# Send POST request
response = requests.post(url, json=payload, headers=headers)

# Handle response
if response.status_code == 200:
    try:
        data = response.json()  # Parse response JSON
        if "response" in data:
            local_response = textwrap.fill(data["response"], width=80)
            print("Generated Summary:\n")
            print(local_response)
        else:
            print("No 'response' key found in the JSON.")
    except json.JSONDecodeError:
        print(f"Invalid JSON response: {response.text}")
else:
    print(f"Error {response.status_code}: {response.text}")

### 5.2.4 Extracting JSON from response

In [ ]:
def extract_json(text):
    # List of regex patterns to try.
    patterns = [
        # Pattern for JSON wrapped in a markdown code block:
        r'```json\s*([\s\S]*?)\s*```',
        # Fallback pattern: JSON object starting with '{' and ending with '}'
        r'({[\s\S]*})'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            json_str = match.group(1)
            # Optionally, remove unwanted control characters.
            json_str = re.sub(r'[\x00-\x1F]+', '', json_str)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError as e:
                print("JSON decode error:", e)
                continue
    return None

local_response = extract_json(local_response)
response_dict = {"model": MODEL_LOCAL, "response": local_response}
local_response

# 6. Cross-model and Iteration Comparison

## 6.1 Overview of results

In [40]:
# Define CSV storage file
CSV_FILE = "pdf_processing_results.csv"

# Define function to append the JSON output of response_json function to a DataFrame
def append_pdf_json_result(pdf_name: str, response_json: dict) -> pd.DataFrame:
    """
    Appends the JSON output of response_json function to a DataFrame.
    
    - If the PDF has been processed before, it appends a **new row** instead of a new column.
    - Prevents duplicate JSON entries for the same iteration.
    - Ensures data is **stored in rows**, making querying and analysis easier.

    Args:
        pdf_name (str): Name of the processed PDF file.
        response_json (dict): JSON response from the knowledge extraction process.

    Returns:
        pd.DataFrame: Updated DataFrame with the new result.
    """
    # Convert JSON response to a formatted string for easy comparison
    json_output = json.dumps(response_json, indent=2)

    # Load existing results if the CSV exists
    path = "./data/" + CSV_FILE
    if os.path.exists(path):
        df = pd.read_csv(path, dtype={"iteration_number": int})
    else:
        # Create an empty DataFrame with the correct schema
        df = pd.DataFrame(columns=["pdf_name", "model_type", "iteration_number", "json_output"])

    model_type = MODEL_GPT if response_dict.get("model") == MODEL_GPT else MODEL_LOCAL

    # Filter for the current PDF's past records
    pdf_history = df[df["pdf_name"] == pdf_name]

    # Check for duplicates: If this JSON output already exists for the same PDF, skip re-adding
    if not pdf_history.empty and json_output in pdf_history["json_output"].values:
        print(f"No changes detected in JSON for {pdf_name}, skipping new entry.")
        return df  # Exit early if it's a duplicate

    # Determine new iteration number
    iteration_number = pdf_history["iteration_number"].max() + 1 if not pdf_history.empty else 1

    # Append new result
    new_entry = pd.DataFrame({"pdf_name": [pdf_name], "model_type": [model_type], "iteration_number": [iteration_number], "json_output": [json_output]})
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save back to CSV in **append mode** to avoid full file reads/writes
    df.to_csv(path, index=False)

    print(f"Successfully added {pdf_name} - Iteration {iteration_number} to results!")
    return df

# Example execution
results_df = append_pdf_json_result(pdf_name, response_dict["response"])

Successfully added IE-6218-200111 Collision RRME Rosslare.pdf - Iteration 2 to results!


In [41]:
results_df

,pdf_name,model_type,iteration_number,json_output
0,IE-10375 - 210827 Collision with track equipme...,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
1,IE-10397 - 211207 Clontarf.pdf,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Incident ..."
2,IE-6218-200111 Collision RRME Rosslare.pdf,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Train A60..."
3,IE-6218-200111 Collision RRME Rosslare.pdf,gpt-4o-mini,2,"{\n ""nodes"": [\n {\n ""id"": ""Train A60..."


## 6.2 Mapping entities to graph nods and rels

In [33]:
# Defining functions for mapping extracted entities to graph nodes and relationships
def props_to_dict(props) -> dict:
    """Converts properties to a dictionary for graph storage."""
    properties = {}
    if not props:
        return properties
    for p in props:
        properties[p["key"]] = p["value"]
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Maps extracted entities to graph nodes."""
    properties = {"name": node.id}
    return BaseNode(
        id=node.id,
        type=node.type.capitalize(),
        properties=properties
    )

def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Maps extracted relationships to graph edges."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}

    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

# 7. Neo4j Storage

## 7.1 Instantiating Neo4j

In [42]:
# Neo4j Connection Setup
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

try:
    # Test the connection
    with driver.session() as session:
        session.run("RETURN 1")
    print("Connected to Neo4j successfully.")
except AuthError as e:
    print("Authentication failed. Check your credentials:", str(e))

Connected to Neo4j successfully.


In [43]:
# Clear database
def clear_neo4j_database():
    """Delete all nodes and relationships in the Neo4j database."""
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("Neo4j database cleared successfully.")

# Run the function to clear the database
clear_neo4j_database()

Neo4j database cleared successfully.


In [44]:
results_df

,pdf_name,model_type,iteration_number,json_output
0,IE-10375 - 210827 Collision with track equipme...,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
1,IE-10397 - 211207 Clontarf.pdf,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Incident ..."
2,IE-6218-200111 Collision RRME Rosslare.pdf,gpt-4o-mini,1,"{\n ""nodes"": [\n {\n ""id"": ""Train A60..."
3,IE-6218-200111 Collision RRME Rosslare.pdf,gpt-4o-mini,2,"{\n ""nodes"": [\n {\n ""id"": ""Train A60..."


## 7.2 Extracting the JSON from DF row

In [ ]:
def get_json_output(df, pdf_name, iteration_number):
    """
    Gets the 'json_output' for the given pdf_name and iteration_number.
    Returns an empty dict if there's no match.
    """
    subset = df[
        (df["pdf_name"] == pdf_name) &
        (df["iteration_number"] == iteration_number)
    ]

    if subset.empty:
        print("No match found.")
        return {}

    json_str = subset.iloc[0]["json_output"]
    return json.loads(json_str)

# Choose the JSON to convert to graph
pdf_of_choice = "IE-6218-200111 Collision RRME Rosslare.pdf"
json_to_convert = get_json_output(results_df, pdf_of_choice, 2)
print(json.dumps(json_to_convert, indent=2))

{
  "nodes": [
    {
      "id": "Train A602 Collision",
      "type": "UniqueAccident"
    },
    {
      "id": "Collision of one or more rail vehicles with obstacle within the clearance gauge",
      "type": "AccidentType"
    },
    {
      "id": "112 MP 880 yards to 112 MP 1320 yards",
      "type": "TrackSection"
    },
    {
      "id": "11/01/2020",
      "type": "Date"
    },
    {
      "id": "10:52",
      "type": "Time"
    },
    {
      "id": "Ireland",
      "type": "Country"
    },
    {
      "id": "Railway Accident Investigation Unit",
      "type": "RegulatoryBody"
    }
  ],
  "rels": [
    {
      "source": "Train A602 Collision",
      "target": "Ireland",
      "type": "occurred_in"
    },
    {
      "source": "Train A602 Collision",
      "target": "Collision of one or more rail vehicles with obstacle within the clearance gauge",
      "type": "is_type"
    },
    {
      "source": "Train A602 Collision",
      "target": "112 MP 880 yards to 112 MP 1320 yards",


## 7.3 Converting the JSON to graph 

In [46]:
def convert_json_to_graph(json_to_convert, source_text):
    """
    Converts extracted JSON into a graph-compatible format with correct entity types.
    """

    def get_node_type(json_data, node_id):
        """
        Helper function to retrieve the correct node type from JSON.
        """
        for node in json_data["nodes"]:
            if node["id"] == node_id:
                return node["type"]
        return "Unknown"  # Fallback if type is missing

    if not json_to_convert:
        print("No valid data to convert to a graph.")
        return None

    # Convert Nodes
    graph_nodes = [map_to_base_node(Node(id=node["id"], type=node["type"])) for node in json_to_convert["nodes"]]

    # Convert Relationships (Ensure correct types)
    graph_rels = []
    for rel in json_to_convert["rels"]:
        source_node = Node(id=rel["source"], type=get_node_type(json_to_convert, rel["source"]))
        target_node = Node(id=rel["target"], type=get_node_type(json_to_convert, rel["target"]))
        graph_rels.append(map_to_base_relationship(Relationship(source=source_node, target=target_node, type=rel["type"])))

    return GraphDocument(nodes=graph_nodes, relationships=graph_rels, source=Document(page_content=source_text))

## 7.4 Storing the graph in Neo4j

In [47]:
def store_in_neo4j(graph_document):
    """
    Stores extracted knowledge graph into Neo4j with dynamic labels.
    """
    with driver.session() as session:
        # Store nodes with dynamic labels
        for node in graph_document.nodes:
            session.run(f"""
                MERGE (n:{node.type} {{id: $id}})
                ON CREATE SET n.name = $name
            """, id=node.id, name=node.id)

        # Store relationships
        for rel in graph_document.relationships:
            session.run(f"""
                MATCH (s {{id: $source}})
                MATCH (t {{id: $target}})
                MERGE (s)-[r:{rel.type.upper()}]->(t)
            """, source=rel.source.id, target=rel.target.id)

In [48]:
def process_railway_accident_report(json_to_convert):
    """
    Converts the JSON to a graph format and stores it in Neo4j.
    """
    print("Converting JSON to graph format...")
    graph_document = convert_json_to_graph(json_to_convert, relevant_report_text)

    if graph_document:
        print("Graph structure created! Storing in Neo4j...")
        store_in_neo4j(graph_document)

In [49]:
# Store extracted entities into Neo4j
try:
    db_result = process_railway_accident_report(json_to_convert)
    print("Data stored in Neo4j successfully.")
except Exception as e:
    print("Failed to store data in Neo4j:", str(e))

Converting JSON to graph format...
Graph structure created! Storing in Neo4j...
Data stored in Neo4j successfully.


In [ ]:
# Close Neo4j connection
driver.close()

# 8. Comparison against ERAIL DB

## 8.1 Loading ERAIL DB

In [ ]:
# Load the ERAIL DB
erail_db = pd.read_excel("erail database.xlsx")
erail_db.head(5)

## 8.2 Pre-processing data

In [ ]:
# Convert date column to datetime & adjust format
erail_db["Date of occurrence"] = erail_db["Date of occurrence"].dt.strftime("%d/%m/%Y")

# Convert time column to datetime & adjust format
erail_db["Time of occurrence"] = pd.to_datetime(erail_db["Time of occurrence"], errors='coerce')
erail_db["Time of occurrence"] = erail_db["Time of occurrence"].dt.strftime("%H:%M")

erail_db.head(5)

## 8.3 Merging data for comparison

In [ ]:
# Create a copy of the results DataFrame
comparison_df = results_df[["pdf_name", "json_output"]].copy()

# Extract ID from the PDF name
comparison_df["ERAIL Occurrence"] = comparison_df["pdf_name"].str.extract(r'(IE-\d+)')

# Sample DataFrame (assuming json_output column contains dictionaries in string format)
comparison_df['json_output'] = comparison_df['json_output'].apply(json.loads)  # Convert JSON string to dictionary

# Function to extract node data
def extract_nodes(json_data):
    node_dict = {}
    for node in json_data.get("nodes", []):
        node_dict[f"gpt_{node['type']}"] = node["id"]  # Store ID based on type
    return pd.Series(node_dict)  # Convert dictionary to Series for easier DataFrame merging

# Apply the function to extract node data
nodes_df = comparison_df['json_output'].apply(extract_nodes)

# Merge extracted data into original DataFrame
comparison_df = pd.concat([comparison_df, nodes_df], axis=1)

# Drop the original json_output column if no longer needed
comparison_df.drop(columns=["json_output"], inplace=True)

# View comparison DataFrame
comparison_df

In [ ]:
# Merge the comparison DataFrame with the ERAIL database
merged_df = comparison_df.merge(erail_db, on="ERAIL Occurrence", how='inner')
merged_df

## 8.4 Comparison

In [ ]:
# Compare GPT extracted date with the ERAIL database (source of truth)
merged_df["Date Match"] = np.where(merged_df["gpt_Date"] == merged_df["Date of occurrence"], "Match", "Mismatch")
merged_df[["ERAIL Occurrence", "gpt_Date", "Date of occurrence", "Date Match"]]

In [ ]:
# Compare GPT extracted time with the ERAIL database (source of truth)
merged_df["Time Match"] = np.where(merged_df["gpt_Time"] == merged_df["Time of occurrence"], "Match", "Mismatch")
merged_df[["ERAIL Occurrence", "gpt_Time", "Time of occurrence", "Time Match"]]

In [ ]:
# Compare GPT extracted country with the ERAIL database (source of truth)
merged_df["Country Match"] = np.where(merged_df["gpt_Country"] == merged_df["Country"], "Match", "Mismatch")
merged_df[["ERAIL Occurrence", "gpt_Country", "Country", "Country Match"]]